In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer

In [4]:
from pyspark.sql.functions import udf

In [5]:
from pyspark.sql.types import IntegerType

In [6]:
sen_df = spark.createDataFrame([
    
    (0,'Hi i heard about Spark'),
    (1,'I wish java could use case classes'),
    (2,'Logistic,regression,modles,are,neat')
],['id','sentence'])

In [7]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi i heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [8]:
#create the tokizer object 
tokinezer  = Tokenizer(inputCol = 'sentence',outputCol='words')

In [9]:
regrex_toknizer = RegexTokenizer(inputCol = 'sentence',outputCol='words',
                                pattern = '\\W')

In [10]:
count_tokens = udf(lambda words : len(words),IntegerType())

In [11]:
tokenized = tokinezer.transform(sen_df)

In [14]:
tokenized.show()
# there is a bit of confusion regarding the last row, could be 
# -- one long string or actually separated so let use the udf to 
# -- count the tokens
tokenized = tokenized.withColumn('token_count',count_tokens(tokenized.words))

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi i heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [15]:
# so its a one long string, need to use the regex tokenizer better
tokenized.show()

+---+--------------------+--------------------+-----------+
| id|            sentence|               words|token_count|
+---+--------------------+--------------------+-----------+
|  0|Hi i heard about ...|[hi, i, heard, ab...|          5|
|  1|I wish java could...|[i, wish, java, c...|          7|
|  2|Logistic,regressi...|[logistic,regress...|          1|
+---+--------------------+--------------------+-----------+



In [16]:
regex_tokenized = regrex_toknizer.transform(sen_df)

In [18]:
regex_tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi i heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic, regres...|
+---+--------------------+--------------------+



In [20]:
regex_tokenized = regex_tokenized.withColumn('tokens_count',count_tokens(regex_tokenized.words))

In [21]:
regex_tokenized.show()

+---+--------------------+--------------------+------------+
| id|            sentence|               words|tokens_count|
+---+--------------------+--------------------+------------+
|  0|Hi i heard about ...|[hi, i, heard, ab...|           5|
|  1|I wish java could...|[i, wish, java, c...|           7|
|  2|Logistic,regressi...|[logistic, regres...|           5|
+---+--------------------+--------------------+------------+



In [22]:
# remove stop words(words that appear frequently and add no value)
from pyspark.ml.feature import StopWordsRemover

In [23]:
sentenceDataframe = spark.createDataFrame([
    (0,['i','saw','the','green','horse']),
    (1,['Mary','had','a','little','lamb'])
],['id','tokens'])

In [24]:
sentenceDataframe.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[i, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [26]:
remover = StopWordsRemover(inputCol = 'tokens',outputCol = 'filterd')

In [27]:
remover.transform(sentenceDataframe).show()

+---+--------------------+--------------------+
| id|              tokens|             filterd|
+---+--------------------+--------------------+
|  0|[i, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [28]:
#n-gram
from pyspark.ml.feature import NGram

In [29]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

In [30]:
ngram = NGram(n = 2,inputCol = 'words',outputCol = 'grams')

In [32]:
ngram.transform(wordDataFrame).select('grams').show(truncate = False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+

